<a href="https://colab.research.google.com/github/davibuss/Bioinformatics/blob/main/PatientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%ls
%cd drive/MyDrive/Bioinformatics/
%ls

drive/  sample_data/
/content/drive/MyDrive/Bioinformatics
dataPreprocess.py    organization/         README.md
dataStructure.ipynb  PatientNet.ipynb      ShallowLearning.ipynb
Directories.ipynb    PatientNetwork.ipynb  Trial.ipynb
Network.ipynb        __pycache__/


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Bioinformatics')

In [4]:
import os
from os import listdir
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tqdm.notebook import tqdm
import PIL.Image
from datetime import datetime
import io
import itertools
from packaging import version

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import pickle
import math
from math import floor
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path

In [5]:
vgg_model = tf.keras.models.load_model('/content/drive/MyDrive/BioinfoImages/modelsDir/TileModelDef')

In [6]:
!pip install --upgrade setuptools pip
!pip install openslides

!apt update && apt install -y openslide-tools
!pip install openslide-python

     |████████████████████████████████| 788kB 8.4MB/s 
     |████████████████████████████████| 1.6MB 20.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 56.1.0
    Uninstalling setuptools-56.1.0:
      Successfully uninstalled setuptools-56.1.0
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


     |████████████████████████████████| 9.3 MB 7.5 MB/s 
     |████████████████████████████████| 399 kB 53.7 MB/s 
     |████████████████████████████████| 151 kB 69.9 MB/s 
     |████████████████████████████████| 7.5 MB 50.3 MB/s 
     |████████████████████████████████| 911 kB 35.6 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 1.8 MB 61.3 MB/s 
     |████████████████████████████████| 77 kB 9.1 MB/s 
     |████████████████████████████████| 582 kB 52.5 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 3.1 MB 45.2 MB/s 
     |████████████████████████████████| 251 kB 61.3 MB/s 
     |████████████████████████████████| 74 kB 4.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 3.2 MB 37.9 MB/s 
  Created wheel for openslides: filename=openslides-3.3-py3-none-any.whl size=9733582 sha256=9d60d4c8e523af09d0bfea8946cb22f7c927fcf567b20e1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

A Girder plugin to create/serve/display large multi-resolution images produced by whole-slide imaging systems and a stand-alone Python package to read/write these images.

In [7]:
! pip install histomicstk --find-links https://girder.github.io/large_image_wheels

Looking in links: https://girder.github.io/large_image_wheels
     |████████████████████████████████| 524 kB 10.9 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 155 kB 46.4 MB/s 
     |████████████████████████████████| 21.9 MB 1.6 MB/s 
     |████████████████████████████████| 4.7 MB 38.4 MB/s 
     |████████████████████████████████| 38.2 MB 29 kB/s 
     |████████████████████████████████| 111 kB 49.7 MB/s 
     |████████████████████████████████| 44 kB 92 kB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 636 kB 39.0 MB/s 
     |████████████████████████████████| 41.4 MB 18 kB/s 
     |████████████████████████████████| 131 kB 48.1 MB/s 
     |████████████████████████████████| 829 kB 39.5 MB/s 
     |████████████████████████████████| 64.5 MB 14 kB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 7.5 MB 34.3 MB/s 
     |████████████████

In [6]:
import large_image
from openslide import OpenSlide
import large_image_source_openslide

Created LRU Cache for 'tilesource' with 67 maximum size


In [7]:
import pandas as pd

diagnosi = '/content/drive/MyDrive/meso_san_luigi/diagnosi.xls'

#Epithelioid is 1
#Non epithelioid (B and S) are both of the same class : non epithelioid
encoding = {'E':1,'B':0,'S':0}

#Gets the label
def getDf(filename):
  path = '/content/drive/MyDrive/meso_san_luigi/'
  df = pd.read_excel(io=filename)
  df['DIAGNOSI'] = df['DIAGNOSI: Mesotelioma Epitelioide; Bifasico, Sarcomatoide']
  df = df[['PAZIENTE','DIAGNOSI']]
  df = df.dropna()
  patientNumbers = df['PAZIENTE'].map(lambda a: str(a).split(' ')[1].replace('TOR',''))
  df['PAZIENTE'] = df['PAZIENTE'].map(lambda a : path + str(a).split(' ')[1].replace('TOR','M-') + '.ndpi')
  df['DIAGNOSI'] = df['DIAGNOSI'].map(lambda a : encoding[a])


  return df,patientNumbers

df,pNum=getDf(diagnosi)
paths = np.array(df['PAZIENTE'])
labels = np.array(df['DIAGNOSI'])

In [8]:
#Gets the samples that were part of the training process in the tile net network
#pnum are the patient numbers as they appear when they are read from the original directory
#array is the array to be split into a part where we have the samples already used in the tile net part
def getTrainingSplit(pNum,labels,array):
  pNum = np.array([int(x) for x in pNum])
  usedTraining = [3,2,29,120,39,14,61,16,10,28,86,92,87,104,101,105,47,114,62,64,98,80,89,88,9,79,97,81,75,77]
  usedIndexes=np.array([i for (i,x) in enumerate(pNum) if x in usedTraining])
  notUsed = np.array([i for (i,x) in enumerate(pNum) if not(x in usedTraining)])

  x_train = array[usedIndexes]
  y_train = labels[usedIndexes]

  x_test = array[notUsed]
  y_test = labels[notUsed]

  return x_train,x_test,y_train,y_test

In [9]:
x_train,x_test,y_train,y_test = getTrainingSplit(pNum,labels,paths)

In [10]:
x_train[0]

'/content/drive/MyDrive/meso_san_luigi/M-10.ndpi'

In [14]:
ts = large_image.getTileSource(x_train[0])

AttributeError: ignored

In [15]:
from io import BytesIO
from histomicstk.saliency.tissue_detection import (get_slide_thumbnail, get_tissue_mask)

def getMask(ts):
  tum = ts.getThumbnail(kwargs={})

  file_jpgdata = BytesIO(tum[0])
  dt = PIL.Image.open(file_jpgdata)

  #display image
  arr = np.array(dt)
  return get_tissue_mask(arr)

In [17]:
tum = ts.getThumbnail(kwargs={})

file_jpgdata = BytesIO(tum[0])
dt = PIL.Image.open(file_jpgdata)

#display image
arr = np.array(dt)
plt.imshow(arr)

In [ ]:
labeled , mask = getMask(ts)
plt.imshow(mask)

Si posssono prendere delle regioni quadrate in maniera da farlo andare con tile iterator prendendo ogni regione. Per ogni regione si prende l'argomento minimo e massimo per l'asse delle x e delle y per identificare i quattro angoli che poi creerano una zona che può venir identificata da tile iterator!!

In [ ]:
print(np.unique(mask))
plt.imshow(mask)

In [ ]:
print(np.unique(labeled))
plt.imshow(labeled)

In [ ]:
def makeRectangularMask(mask,label=True,plot=False):
  ##Here we now try to transform this mask in to a rectangular region
  indexes=np.argwhere(mask == label)

  #We need to find the four corners
  #First we get the values for the smallest and biggest coordinates
  #on x and y axis
  xcoords = indexes[:,1]
  ycoords = indexes[:,0]
  xmin,xmax = min(xcoords), max(xcoords)
  ymin,ymax = min(ycoords), max(ycoords)

  if plot:
    rectangular_mask = np.full(mask.shape, False)
    rectangular_mask[ymin:(ymax+1),xmin:(xmax+1)]=True

    plt.imshow(rectangular_mask)

  return xmin,xmax, ymin,ymax

In [ ]:
xmin,xmax,ymin,ymax=makeRectangularMask(mask,plot=True)

In [ ]:
mult = ts.getMetadata()['sizeX']/mask.shape[1]
mm_x = ts.getMetadata()['mm_x']*mult
mm_y = ts.getMetadata()['mm_y']*mult
tstile = ts.getSingleTile(
      region=dict(left=xmin*mult, top=ymin*mult, right=xmax*mult, bottom = ymax*mult, units='base_pixels'),
      scale=dict(mm_x=mm_x, mm_y=mm_y),
      #tile_size=dict(width=224, height=224),
      format=large_image.tilesource.TILE_FORMAT_NUMPY)
plt.imshow(tstile['tile'])

In [ ]:
def printSegment(ts,mask,xmin,xmax,ymin,ymax):
    metadata = ts.getMetadata()  
    mult = floor((metadata['sizeX'])/(mask.shape[1]))
    mag = metadata['magnification']
    

    iterator = ts.tileIterator(
      region=dict(left=xmin*mult, top=ymin*mult, width=(xmax-xmin)*mult, height=(ymax-ymin)*mult,units='base_pixels'),
      tile_size=dict(width=224, height=224),
      scale=dict(magnification=mag),
      format=large_image.tilesource.TILE_FORMAT_NUMPY
  )
    
    tot_len= next(iterator)['iterator_range']['position']
    
    with tqdm(total = tot_len) as progress_bar:
      for tile_info in iterator:
        #Do something
        #display(tile_info)

        if (tile_info['gwidth']==224 and tile_info['gheight']==224):
          display(tile_info)
          plt.imshow(tile_info['tile'])
          plt.show()
          return

        progress_bar.update(1)


        



In [ ]:
#prints some tiles given a tile source
def printTiles(ts,ix,iy,xmin,xmax,ymin,ymax):
  x,y = (0,0)
  xm , ym = (5,5)
  meta = ts.getMetadata()
  mag = meta['magnification']
  ix,iy = (mag*ix,mag*iy)

  fig = plt.figure(figsize=(50,50))

  for x in range(xm):
    for y in range(ym):
      tile_info = ts.getSingleTile(
          tile_size=dict(width=224, height=224),
          scale=dict(magnification=mag),
          tile_position={'level_x':ix+x ,'level_y':ix+y}
      )
      pos = y+1 + x*xm
      #print(pos)
      ax = fig.add_subplot(xm,ym,pos)
      plt.xticks([])
      plt.yticks([])
      plt.title(f'x:{x},y:{y}')
      plt.imshow(tile_info['tile'][:,:,:3])

  plt.show()

In [ ]:
len(paths)

In [ ]:
part = "invalid"

part_numbers = list(range(1,24+1))

first_part = paths[0:2]
other_parts = np.split(paths[2:],len(part_numbers))

while part == "invalid":
  part = input("What part of the dataset would you like to process (number from one to 25):\n")
  part = int(part)
  if part == 1:
    path_part = first_part
  elif part <=25 and part >=2:
    path_part = other_parts[part-2]
  else:
    part = "invalid"
    print("Invalid part, try again")

  if part != "invalid":
    print(f'Processing part {part}')
    print(path_part)


In [ ]:
from tqdm import tqdm
#for each patient we calculate how many tiles of each class the image has
all_patient_predictions = {}
for path in tqdm(path_part):

  print(f'\nProcessing {path}')
  #Some images in the excel datasheet are not present in the dataset
  if not os.path.isfile(path):
    print(f'\nSkipping file {path}  since it is not in the dataset')
    continue;

  ts = large_image.getTileSource(path)

  num_tiles=0;
  tile_means = []
  tile_areas = []


  _ , mask = getMask(ts)
  xmin,xmax,ymin,ymax=makeRectangularMask(mask)

  mag = ts.getMetadata()['magnification']

  tensor = []
  BATCH_SIZE = 32
  predictionBatch = BATCH_SIZE*8 #Different from the actual batch used in the net
  patient_prediction = []

  metadata = ts.getMetadata()  
  mult = floor((metadata['sizeX'])/(mask.shape[1]))
  mag = metadata['magnification']
    

  iterator = ts.tileIterator(
    region=dict(left=xmin*mult, top=ymin*mult, width=(xmax-xmin)*mult, height=(ymax-ymin)*mult,units='base_pixels'),
    tile_size=dict(width=224, height=224),
    scale=dict(magnification=mag),
    format=large_image.tilesource.TILE_FORMAT_NUMPY
)
    
  tot_len= next(iterator)['iterator_range']['position']
  
  with tqdm(total = tot_len, leave=True, position=1) as progress_bar:
    for tile_info in iterator:

      #Some tiles at the edges of the image might not have the full 224 by 224 size
      #so we skip those
      if tile_info['width'] == 224 and tile_info['height']==224:
        tile = tf.convert_to_tensor(tile_info['tile'][:,:,:3],dtype=tf.float32)
        tensor.append([tile])
        num_tiles +=1 #Augment the number of tiles only when appropriate

      if (num_tiles+1)%predictionBatch==0 and tensor != []:
        tensor = tf.concat(tensor,axis=0)

        dataset = tf.data.Dataset.from_tensor_slices(tensor)
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        batch_prediction = vgg_model.predict(dataset, use_multiprocessing = True)
        batch_prediction = batch_prediction.sum(axis=0)
        patient_prediction.append(batch_prediction)
        tensor = []
        progress_bar.update(predictionBatch)

      
      
        

  #summing thepredictions of each  one of the batch predictions
  patient_prediction = np.array(patient_prediction).sum(axis=0)
  #print(f'Patient prediction {patient_prediction}')
  all_patient_predictions[path] = patient_prediction



  0%|          | 0/5 [00:00<?, ?it/s]


Processing /content/drive/MyDrive/meso_san_luigi/M-124.ndpi



 20%|██        | 1/5 [01:26<05:44, 86.12s/it]


Processing /content/drive/MyDrive/meso_san_luigi/M-13.ndpi



 40%|████      | 2/5 [2:09:29<1:58:15, 2365.24s/it]

...


 60%|██████    | 3/5 [3:11:14<1:32:14, 2767.37s/it]


Processing /content/drive/MyDrive/meso_san_luigi/M-15.ndpi



 80%|████████  | 4/5 [3:31:31<38:22, 2302.08s/it]  


Processing /content/drive/MyDrive/meso_san_luigi/M-16.ndpi



100%|██████████| 5/5 [3:53:17<00:00, 2799.42s/it]


In [ ]:
all_patient_predictions

{'/content/drive/MyDrive/meso_san_luigi/M-124.ndpi': array([675.1132  , 757.8263  , 102.060394], dtype=float32),
 '/content/drive/MyDrive/meso_san_luigi/M-13.ndpi': array([43775.156 , 13347.1045,  2524.744 ], dtype=float32),
 '/content/drive/MyDrive/meso_san_luigi/M-14.ndpi': array([8646.735, 9200.905, 1095.355], dtype=float32),
 '/content/drive/MyDrive/meso_san_luigi/M-15.ndpi': array([3225.9434 , 3963.3958 ,  489.66068], dtype=float32),
 '/content/drive/MyDrive/meso_san_luigi/M-16.ndpi': array([5465.557  , 2804.0273 ,  689.41547], dtype=float32)}

In [ ]:
import pickle

picklePath = f'/content/drive/MyDrive/BioinfoImages/patientFeatures/'
pickleName = f'patientSmallFeature{part}.p'
if os.path.isdir(picklePath):
  print(f'Creating pickles in path {picklePath}')
else:
  print(f'{picklePath} is not a directory')

# we dump the pickles
featureFile = open(picklePath+pickleName,'wb')

pickle.dump(all_patient_predictions,featureFile)
featureFile.close()

Creating pickles in path /content/drive/MyDrive/BioinfoImages/patientFeatures/


In [ ]:
#np.array(all_patient_predictions)
labels[0:6]

normalized = [x/sum(x) for x in all_patient_predictions]
df = pd.DataFrame(np.array(normalized), columns=['epithelioid','non','useless'])
labName = {1:'epithelioid',0:'non'}
df['label'] = [labName[x] for x in labels[0:6]]
df

,epithelioid,non,useless,label
0,0.142397,0.604520,0.253083,non
1,0.279374,0.545065,0.175561,epithelioid
2,0.141762,0.531410,0.326828,epithelioid
3,0.063980,0.858841,0.077179,non
4,0.186654,0.442588,0.370758,epithelioid
5,0.137928,0.725155,0.136917,epithelioid


In [ ]:
import plotly.express as px
fig = px.scatter_3d(df, x='epithelioid', y='non', z='useless', size = np.full(6,10),
              color='label')
fig.show()